In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data_dir = '../data/diamonds.csv'

In [5]:
with open(data_dir, 'rb') as csvfile:
    dataset = pd.read_csv(data_dir)
    
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [4]:
#carat	cut	color	clarity	depth	table	price	x	y	z
print(dataset[''])

0        E
1        E
2        E
3        I
4        J
5        J
6        I
7        H
8        E
9        H
10       J
11       J
12       F
13       J
14       E
15       E
16       I
17       J
18       J
19       J
20       I
21       E
22       H
23       J
24       J
25       G
26       I
27       J
28       D
29       F
        ..
53910    E
53911    E
53912    F
53913    G
53914    I
53915    E
53916    D
53917    J
53918    I
53919    I
53920    E
53921    E
53922    D
53923    I
53924    I
53925    I
53926    E
53927    F
53928    E
53929    G
53930    E
53931    F
53932    E
53933    E
53934    D
53935    D
53936    D
53937    D
53938    H
53939    D
Name: color, Length: 53940, dtype: object
